In [ ]:
import torch
import segmentation_models_pytorch as smp
import yaml
import json
import numpy as np
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
import shapely
from rasterio import features
from shapely.ops import unary_union
from prepare_data import read_polys, poly2mask

In [ ]:
imgs = sorted(Path("/mnt/storage/datasets/walrus/raw/walrusval/images/").glob("*.jpg"))
preds = sorted(Path("/home/zakirov/data/full_predict/").glob("*.png"))
trues = sorted(Path("/mnt/storage/datasets/walrus/raw/walrusval/markup/").glob("*.json"))
trues.pop(trues.index(Path("/mnt/storage/datasets/walrus/raw/walrusval/markup/categories.json")));

In [ ]:
plt.figure(figsize=(30, 15))
_ii = cv2.imread(str(imgs[5]))[..., ::-1]
# _ii = np.flipud(_ii)
plt.imshow(_ii);

In [ ]:
MIN_AREA = 100 # min size based on training data
def pred2poly(pred):
    morj_no_border = np.clip(pred[..., 2] - pred[..., 1] - pred[..., 0], 0, 1)
    morj_thr = (morj_no_border > 0.5).astype(np.uint8)
    morj_mask = morj_thr
    pred_polys = [shapely.geometry.shape(sh) for sh, _ in features.shapes(morj_mask, mask=morj_mask)]
    pred_polys = [p for p in pred_polys if p.area > MIN_AREA]
    return pred_polys

In [ ]:
GLOBAL_TP, GLOBAL_FP, GLOBAL_FN = 0, 0, 0
preds_cp = preds
trues_cp = trues
preds_cp = preds# [:6] # + preds[6:] 
trues_cp = trues#[:6] #+ trues[6:] 
GLOBAL_SCORE = 0

for pr, tr in zip(preds_cp, trues_cp):
    prr = cv2.imread(str(pr))[:, :, ::-1].astype(float) / 255
    pred_polys = pred2poly(prr)
    true_polys = read_polys(tr)
    
    found_true_idx = {}
    TP, FP, FN = 0, 0, 0
    found_pred = set()
    found_true = set()
    for true_idx, true_p in enumerate(true_polys):
        found = False
        found_second_time = 0
        for pred_idx, pred_p in enumerate(pred_polys):
            if not true_p.contains(pred_p.centroid):
                continue
            if found:
                found_second_time += 1
                continue
            found = True
            found_pred.add(pred_idx)
        if found:
            found_true.add(true_idx)
    not_morj = len(pred_polys) - len(found_pred) - found_second_time
    GLOBAL_SCORE += len(found_true) - found_second_time - not_morj * 2
    FN = len(true_polys) - len(found_true)
    FP = len(pred_polys) - len(found_pred)
    TP = len(found_true)
    recall = TP / (TP + FN)
    prec = TP / (TP + FP)
    f_score = 2 * prec * recall / (prec + recall)
    print(f"Prec: {prec:.2f}. Recall: {recall:.2f}. F1: {f_score:.2f}. N true: {len(true_polys)}")
    GLOBAL_TP += TP
    GLOBAL_FP += FP
    GLOBAL_FN += FN
    
recall = GLOBAL_TP / (GLOBAL_TP + GLOBAL_FN)
prec = GLOBAL_TP / (GLOBAL_TP + GLOBAL_FP)
f_score = 2 * prec * recall / (prec + recall)
print(f"Global Prec: {prec:.3f}. Global Recall: {recall:.3f}. Global F1: {f_score:.3f}")
print(f"Global Score: {GLOBAL_SCORE / (GLOBAL_TP + GLOBAL_FN):.3f}")

In [15]:
Prec: 0.99. Recall: 0.97. F1: 0.98. N true: 224
Prec: 0.96. Recall: 0.83. F1: 0.89. N true: 53
Prec: 0.90. Recall: 0.95. F1: 0.93. N true: 176
Prec: 0.88. Recall: 0.90. F1: 0.89. N true: 177
Prec: 0.95. Recall: 0.98. F1: 0.96. N true: 42
Prec: 0.96. Recall: 0.35. F1: 0.51. N true: 187
Prec: 0.97. Recall: 0.96. F1: 0.97. N true: 76
Prec: 1.00. Recall: 1.00. F1: 1.00. N true: 65
Prec: 0.96. Recall: 0.73. F1: 0.83. N true: 30
Prec: 0.96. Recall: 0.99. F1: 0.97. N true: 209
Prec: 0.97. Recall: 0.99. F1: 0.98. N true: 112
Prec: 0.94. Recall: 0.98. F1: 0.96. N true: 52
Prec: 0.92. Recall: 1.00. F1: 0.96. N true: 48
Prec: 0.97. Recall: 0.97. F1: 0.97. N true: 279
Prec: 0.97. Recall: 0.98. F1: 0.98. N true: 286
Prec: 0.94. Recall: 0.78. F1: 0.85. N true: 276
Prec: 0.99. Recall: 0.99. F1: 0.99. N true: 204
Prec: 0.90. Recall: 0.87. F1: 0.88. N true: 161
Prec: 0.99. Recall: 0.99. F1: 0.99. N true: 323
Prec: 1.00. Recall: 1.00. F1: 1.00. N true: 173
Prec: 0.97. Recall: 0.99. F1: 0.98. N true: 148
Prec: 0.93. Recall: 0.96. F1: 0.95. N true: 149
Prec: 0.98. Recall: 0.96. F1: 0.97. N true: 226
Prec: 0.98. Recall: 0.97. F1: 0.97. N true: 262
Prec: 0.96. Recall: 1.00. F1: 0.98. N true: 216
Prec: 0.99. Recall: 1.00. F1: 0.99. N true: 219
Prec: 0.97. Recall: 1.00. F1: 0.98. N true: 216
Prec: 0.97. Recall: 1.00. F1: 0.98. N true: 212
Prec: 0.99. Recall: 0.99. F1: 0.99. N true: 211
Prec: 0.86. Recall: 0.98. F1: 0.91. N true: 88
Global Prec: 0.962. Global Recall: 0.939. Global F1: 0.950
Global Score: 0.865  


SyntaxError: invalid syntax (820528053.py, line 1)

In [ ]:
_morj_no_border = np.clip(prr[..., 2] - prr[..., 1] - prr[..., 0], 0, 1)
# _morj_no_border = prr[..., 2] - prr[..., 1] - prr[..., 0]
# morj_thr = (morj_no_border > 0.5).astype(np.uint8)
# morj_mask = morj_thr

In [ ]:
plt.figure(figsize=(30, 15))
plt.imshow(prr[..., 0]);

In [ ]:
pred_polys